In [2]:
import numpy as np #linear algebra
import pandas as pd #data processing

import os
import re
import nltk


In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

In [4]:
train_data=pd.read_csv('./train.csv')
test_data=pd.read_csv('./test.csv')

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print(train_data.isnull().sum())
print('************')
print(test_data.isnull().sum())
 


In [ ]:
test_data=test_data.fillna(' ')
train_data=train_data.fillna(' ')
test_data['total']=test_data['title']+' '+test_data['author']+test_data['text']
train_data['total']=train_data['title']+' '+train_data['author']+train_data['text']

# Creating Workcloud Visuals

In [ ]:
real_words = ''
fake_words = ''
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in train_data[train_data['label']==1].text: 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    real_words += " ".join(tokens)+" "

for val in train_data[train_data['label']==0].text: 
      
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    fake_words += " ".join(tokens)+" "

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(fake_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(real_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
# Drop rows with NaN values in the 'text' column
train_data = train_data.dropna(subset=['text'])
test_data = test_data.dropna(subset=['text'])

# Cleaning And PreProcessing

# 1 REGEX

In [ ]:
#Remove punctuations from the String  
s = "!</> hello $$ </>^this is !!!my %f%i&rst&% $@@@project^^^&&!& </>*@# &&\ @@@##%^^&!@# %%$"
s = re.sub(r'[^\w\s]','',s)
print(s)

In [ ]:
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return cleaned_text

# 2 Tokenization

In [ ]:
nltk.word_tokenize("Hello how are you")

# 3 StopWords

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words)

In [ ]:
sentence = "Covid-19 pandemic has impacted many countries and what it did to economy is very stressful "
words = nltk.word_tokenize(sentence)
words = [w for w in words if w not in stop_words]
words

# 4 Lemmatization


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
input_str="been had done languages cities mice"

# tokenize the sentence 
input_str=nltk.word_tokenize(input_str)

#Lemmatization each word
for word in input_str:
    print(lemmatizer.lemmatize(word))


# Apply

In [ ]:
lemmatizer=WordNetLemmatizer()
for index,row in train_data.iterrows():
    filter_sentence = ''
    
    sentence = row['total']
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train_data.loc[index,'total'] = filter_sentence


In [ ]:
train = train_data[['total','label']]

# Applying NLP techniques


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Tokenization, lowercase, and remove stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
train_data['processed_text'] = train_data['text'].apply(preprocess_text)
test_data['processed_text'] = test_data['text'].apply(preprocess_text)

# Bag of Words/ CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())

In [ ]:
print(X.toarray())

In [ ]:
# Feature extraction using count vectorization and tfidf.
count_vectorizer = CountVectorizer()
freq_term_matrix = count_vectorizer.fit_transform(train['total'])

# TF-IDF Vectorization

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train_data['processed_text'])
y_train = train_data['label']

X_test = tfidf_vectorizer.transform(test_data['processed_text'])

# Pickle

In [ ]:
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [ ]:
# Create 'models' directory if it doesn't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

In [ ]:
# Create and train the Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

In [ ]:
# Save the model and TF-IDF vectorizer
model_filepath = os.path.join(models_dir, 'logreg_model.pkl')
vectorizer_filepath = os.path.join(models_dir, 'tfidf_vectorizer.pkl')

In [ ]:
with open(model_filepath, 'wb') as model_file:
    pickle.dump(logreg_model, model_file)

with open(vectorizer_filepath, 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)